In [ ]:
import geopandas as gpd
import numpy as np
import geoviews as gv
import cartopy.crs as ccrs
import hvplot.pandas

from shapely.geometry import LineString, Point
from scipy.interpolate import splev, splrep, BSpline

gv.extension('bokeh',logo=False)

# module 

https://computational-sediment-hyd.hatenablog.jp/entry/2022/05/29/220000

In [ ]:
def chaikins_corner_cutting(coords, refinements=5):
    coords = np.array(coords)

    for _ in range(refinements):
        L = coords.repeat(2, axis=0)
        R = np.empty_like(L)
        R[0] = L[0]
        R[2::2] = L[1:-1:2]
        R[1:-1:2] = L[2::2]
        R[-1] = L[-1]
        coords = L * 0.75 + R * 0.25

    return coords

# main 

In [ ]:
gdf = gpd.read_file('line.geojson', encoding='UTF-8')

In [ ]:
cline = gdf[gdf['name'] == 'Cline']['geometry'].squeeze()
clines = chaikins_corner_cutting(cline.coords[:], 5)
clines = LineString(clines)

lline = gdf[gdf['name'] == 'Lline']['geometry'].squeeze()
llines = chaikins_corner_cutting(lline.coords[:], 5)
llines = LineString(llines)

rline = gdf[gdf['name'] == 'Rline']['geometry'].squeeze()
rlines = chaikins_corner_cutting(rline.coords[:], 5)
rlines = LineString(rlines)

In [ ]:
gdft = gdf.copy()

In [ ]:
gdft.loc[(gdft[gdft['name'] == 'Cline'].index[0]),'geometry'] = clines
gdft.loc[(gdft[gdft['name'] == 'Lline'].index[0]),'geometry'] = llines
gdft.loc[(gdft[gdft['name'] == 'Rline'].index[0]),'geometry'] = rlines

In [ ]:
out = gdft.to_file("linesmooth.geojson", driver='GeoJSON')
del out

#  check

In [ ]:
g0 = gdf.hvplot(label='initial')
g1 = gdft.hvplot(label='smoothing')

back = gv.WMTS('https://mt1.google.com/vt/lyrs=s&x={X}&y={Y}&z={Z}', name="GoogleMapsImagery")
gall = g0*g1*back
gallo = gall.options(width=600,height=500)

In [ ]:
gallo